In [2]:
import sys
import os
from pathlib import Path

# importディレクトリの追加
# sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
print(sys.path)

# プロキシの設定
# os.environ['HTTP_PROXY'] = ''
# os.environ['HTTPS_PROXY'] = ''

%matplotlib inline

['/home/y-katayama/notebooks/dl_study/02_pytorch_tutorial', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/y-katayama/venv/pt1.7/lib/python3.8/site-packages']


In [3]:
!nvidia-smi

Fri Dec  9 14:43:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 40%   34C    P8    17W / 184W |     50MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
| 40%   

In [4]:
from __future__ import print_function
import torch

# Tensor

- 初期化されていない行列は実際に使用されるまで、値は不定となる

In [5]:
# 未初期化のテンソルを作成
x = torch.empty(5, 3)

print(x)  # => 色んな値が入ってる

tensor([[6.3167e-14, 4.5688e-41, 6.3166e-14],
        [4.5688e-41, 6.3166e-14, 4.5688e-41],
        [6.2012e-14, 4.5688e-41, 6.3166e-14],
        [4.5688e-41, 6.2010e-14, 4.5688e-41],
        [6.2010e-14, 4.5688e-41, 6.2014e-14]])


In [6]:
# 乱数によって初期化されたテンソル
x = torch.rand(5, 3)
print(x)

# dtypeパラメータで行列の型を指定できる
y = torch.zeros(5, 3, dtype=torch.long)
print(y)

# 数値を指定して生成することもできる
z = torch.tensor([5.5, 3])
print(z)

tensor([[0.4946, 0.8260, 0.3482],
        [0.2733, 0.0091, 0.4329],
        [0.2194, 0.8463, 0.7269],
        [0.5101, 0.9753, 0.4012],
        [0.5884, 0.9438, 0.1231]])
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])
tensor([5.5000, 3.0000])


# テンソルの形状

- テンソルの形状は`.shape`または`.size()`で取得できる
- torch.Size()はタプルなので, 注意すること

In [7]:
print('x.shape: ', x.shape)
print('x.size(): ', x.size())

x.shape:  torch.Size([5, 3])
x.size():  torch.Size([5, 3])


# テンソルの層さ(変形・変換等)

- tensorの形を変えたい場合は`torch.view`を使用する
  - numpyのreshapeに相当するが, 微妙に機能が異なる

In [8]:
x = torch.randn(4, 4)
print(x)

y = x.view(16)
print(y)

# -1を指定すると他に設定した次元から, 適切な値を自動で計算してくれる
z = x.view(-1, 8)

print(z)

tensor([[-0.1091, -0.4157,  0.8363,  0.7931],
        [-0.3663,  2.1845,  1.4316,  0.8661],
        [-2.3937,  0.0236, -0.1421,  0.8042],
        [-0.5636,  1.1440, -1.3192,  0.4795]])
tensor([-0.1091, -0.4157,  0.8363,  0.7931, -0.3663,  2.1845,  1.4316,  0.8661,
        -2.3937,  0.0236, -0.1421,  0.8042, -0.5636,  1.1440, -1.3192,  0.4795])
tensor([[-0.1091, -0.4157,  0.8363,  0.7931, -0.3663,  2.1845,  1.4316,  0.8661],
        [-2.3937,  0.0236, -0.1421,  0.8042, -0.5636,  1.1440, -1.3192,  0.4795]])


## 残りは0章と重複するので省略

# 2 AutoGrad(自動微分)

- `.requires_grad`が`True`の場合, autogradパッケージによってすべての操作が追跡される
  - requires_gradを指定しない場合は, デフォルトでFalseが指定される

- 追跡履歴からTensorを切り離して追跡を停止する場合は `.detach()`を呼び出す
- 演算が終了した際は`.backward()`を呼び出すことで、すべての操作に対する勾配が自動的に計算される
    - `with torch.no_grad()`ブロック内では, まとめて追跡履歴を省略することができる



In [9]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

y = x + 2
print(y)

y = y.detach()
print(y.requires_grad)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
False


## 誤差逆伝播

In [10]:
del y
y = x + 2

z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [11]:
out.backward()
print(out)

# 勾配を表示
print(x.grad)

tensor(27., grad_fn=<MeanBackward0>)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


- `torch.autograd`はベクトルのヤコビアンの積を算出する
- ベクトルのヤコビアンの積は, スカラ量ではない出力を持つモデルに対して,  
  **外部から異なる勾配を追加して計算**する際に有効
  
- `.backward()`を実行する変数はスカラーでなければならないことに注意する。
    - 2つ以上の要素を持つ配列で`backward()`を実行すると、エラーが発生する。
    - 2つ以上の要素を持つ配列で誤差逆伝播したい場合は, 

In [12]:
x = torch.randn(3, requires_grad=True)
y = x * 2
# print('x: ', x)
# print('y: ', y)

# yのノルムを計算(各要素の二乗和)し, L2: 1000以下なら二乗する
while y.data.norm() < 1000:
    # print(f'handcrafted L2', torch.sqrt(y[0]*y[0] + y[1]*y[1] + y[2]*y[2]).item())
    print(f'norm():', y.data.norm().item())
    # print(f'[{i}] y: ', y)
    y = y * 2
    # i = i + 1

norm(): 1.6952590942382812
norm(): 3.3905181884765625
norm(): 6.781036376953125
norm(): 13.56207275390625
norm(): 27.1241455078125
norm(): 54.248291015625
norm(): 108.49658203125
norm(): 216.9931640625
norm(): 433.986328125
norm(): 867.97265625


## ヤコビアン行列の補足
https://qiita.com/sudominoru/items/bfe5a3f6499d0ecf8143

- このセルの出力から何倍してyを求めたのか分かる。

- `y = scale * x`なので、yのxに関する勾配はscaleとなる
- yの勾配をvに対して計算すると, xに対する勾配値が`x.grad`に格納される
- `x.grad`はscaleに`v＝[0.1, 1, 0.0001]`が乗算された値となる

In [13]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)

# vに対するyの勾配を計算
y.backward(v)

print(x.grad)
print("x：", x)
print("y：", y)

# scaleはyに対するxの倍数
scale = y/x
print("倍数：", scale)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])
x： tensor([ 0.3558, -0.2641, -0.7226], requires_grad=True)
y： tensor([  728.6443,  -540.9526, -1479.8494], grad_fn=<MulBackward0>)
倍数： tensor([2048., 2048., 2048.], grad_fn=<DivBackward0>)


In [14]:
a = x.detach().clone()
print(a.requires_grad)
print(a.eq(x))

False
tensor([True, True, True])
